In [1]:
import yfinance as yf

In [8]:
spxs = yf.Ticker("SPXS")
sds = yf.Ticker("SDS")
sh = yf.Ticker("SH")

In [10]:
spxs.info


{'phone': '866-476-7523',
 'longBusinessSummary': 'The fund, under normal circumstances, invests at least 80% of its net assets (plus borrowing for investment purposes) in swap agreements, futures contracts, short positions or other financial instruments that provide inverse (opposite) or short daily exposure to the index or to ETFs that track the index, which, in combination, provider returns consistent with the fund’s investment objective. The index is a float-adjusted, market capitalization-weighted index. The fund is non-diversified.',
 'maxAge': 86400,
 'priceHint': 2,
 'previousClose': 17.53,
 'open': 17.5,
 'dayLow': 17.4494,
 'dayHigh': 17.755,
 'regularMarketPreviousClose': 17.53,
 'regularMarketOpen': 17.5,
 'regularMarketDayLow': 17.4494,
 'regularMarketDayHigh': 17.755,
 'volume': 17182574,
 'regularMarketVolume': 17182574,
 'averageVolume': 28725901,
 'averageVolume10days': 19809310,
 'averageDailyVolume10Day': 19809310,
 'bid': 17.53,
 'ask': 17.54,
 'bidSize': 28000,
 'a

In [11]:
sds.info


{'longBusinessSummary': "The fund invests in financial instruments that ProShare Advisors believes, in combination, should produce daily returns consistent with the fund's investment objective. The index is a measure of large-cap U.S. stock market performance. The fund is non-diversified.",
 'maxAge': 86400,
 'priceHint': 2,
 'previousClose': 39.71,
 'open': 39.67,
 'dayLow': 39.57,
 'dayHigh': 40.06,
 'regularMarketPreviousClose': 39.71,
 'regularMarketOpen': 39.67,
 'regularMarketDayLow': 39.57,
 'regularMarketDayHigh': 40.06,
 'volume': 4808532,
 'regularMarketVolume': 4808532,
 'averageVolume': 7474041,
 'averageVolume10days': 5378440,
 'averageDailyVolume10Day': 5378440,
 'bid': 39.68,
 'ask': 39.73,
 'bidSize': 800,
 'askSize': 2900,
 'yield': 0.0108,
 'totalAssets': 978468160,
 'fiftyTwoWeekLow': 38.15,
 'fiftyTwoWeekHigh': 57.07,
 'fiftyDayAverage': 41.653,
 'twoHundredDayAverage': 44.281,
 'navPrice': 39.7193,
 'currency': 'USD',
 'category': 'Trading--Inverse Equity',
 'ytdRe

In [12]:
sh.info

{'longBusinessSummary': "The fund invests in financial instruments that ProShare Advisors believes, in combination, should produce daily returns consistent with the fund's investment objective. The index is a measure of large-cap U.S. stock market performance. It is a float-adjusted, market capitalization-weighted index of 500 U.S. operating companies and real estate investment trusts selected through a process that factors in criteria such as liquidity, price, market capitalization, financial viability and public float. The fund is non-diversified.",
 'maxAge': 86400,
 'priceHint': 2,
 'previousClose': 14.97,
 'open': 14.96,
 'dayLow': 14.94,
 'dayHigh': 15.0293,
 'regularMarketPreviousClose': 14.97,
 'regularMarketOpen': 14.96,
 'regularMarketDayLow': 14.94,
 'regularMarketDayHigh': 15.0293,
 'volume': 28626169,
 'regularMarketVolume': 28626169,
 'averageVolume': 33070352,
 'averageVolume10days': 27936070,
 'averageDailyVolume10Day': 27936070,
 'bid': 14.95,
 'ask': 14.97,
 'bidSize'